In [1]:
import syft as sy

In [2]:
import torch as th

In [3]:
domain = sy.login(port=8081, email="info@openmined.org", password="changethis")


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into canada... done!


In [4]:
# due to our authorization model allowing any user to `EXECUTE` on any data
# item, a guest user can mutate any object in the store even if it:
# - does not belong to them but is visible
# - does not belong to them and is not visible but the UID can be guessed
# - combining with data they own will make the object and down stream
#   derived objects partially owned by the guest and the original owner
#   will no longer be able to .get their own objects any longer

In [4]:
# admin store
domain.store

""


In [5]:
x = th.tensor([1, 2, 3])
x_ptr = x.send(domain, pointable=True, tags=["visible"])
x_ptr.id_at_location

<UID: 43206751df754a6f9e6c463b20eb64d0>

In [6]:
y = th.tensor([3, 6, 9])
y_ptr = y.send(domain, pointable=False, tags=["invisible"])
y_ptr.id_at_location

<UID: 2b1f6d3c375542ceb5f70de6da91e96b>

In [7]:
domain.store

,ID,Tags,Description,object_type
0,<UID: 43206751df754a6f9e6c463b20eb64d0>,[visible],,<class 'torch.Tensor'>


In [8]:
guest = sy.login(port=8081)

Connecting to None... done! 	 Logging into canada... as GUEST...done!


In [9]:
guest.store

,ID,Tags,Description,object_type
0,<UID: 43206751df754a6f9e6c463b20eb64d0>,[visible],,<class 'torch.Tensor'>


In [10]:
# this is considered allowed
guest_x = guest.store[x_ptr.id_at_location]

In [11]:
try:
    guest_x.get(delete_obj=False)
except Exception:
    print("this should fail as expected")

[2022-02-21T06:38:56.183793-0500][CRITICAL][logger]][276922] You do not have permission to .get() Object with ID: <UID: 43206751df754a6f9e6c463b20eb64d0> on node canada Please submit a request.


this should fail as expected


In [12]:
# this is technically allowed but has bad consequences due to mutation
guest_x.add_(guest_x)

<TensorPointer -> canada:a21ea12beb894207a25ba2100ca44f28>

In [13]:
# this UID is not visible and cant be accessed directly
try:
    guest_x = guest.store[y_ptr.id_at_location]
except Exception:
    print("this should fail")

this should fail


In [14]:
print("if the guest can guess this UID though:")
str(y_ptr.id_at_location.no_dash)

if the guest can guess this UID though:


'2b1f6d3c375542ceb5f70de6da91e96b'

In [15]:
from copy import copy

In [16]:
guest_y = copy(guest_x)

In [17]:
# they can simply construct a pointer client side and send the op
guest_y.id_at_location = sy.common.UID.from_string(
    y_ptr.id_at_location.no_dash
)

In [18]:
guest_y.id_at_location

<UID: 2b1f6d3c375542ceb5f70de6da91e96b>

In [19]:
guest_y.add_(guest_y)

<TensorPointer -> canada:95351c5dd4c8422b95f930104c698f08>

In [21]:
# lets see the results

In [20]:
# this should be 1, 2, 3 but it has been mutated with an add_ by guest
x_ptr.get(delete_obj=False)

tensor([1, 2, 3])

In [21]:
# this should be 3, 6, 9 but it has been mutated with an add_ by guest
y_ptr.get(delete_obj=False)

tensor([3, 6, 9])

In [22]:
# if an object cannot be gotten we can deserialize it from the db
# here to inspect it manually

In [23]:
hex_str = """
0A21737966742E77726170706572732E746F7263682E54656E736F7257726170706572121B0A1222090A0103A20103060C123205696E74363422050A03637075
""".strip()
print(hex_str)

0A21737966742E77726170706572732E746F7263682E54656E736F7257726170706572121B0A1222090A0103A20103060C123205696E74363422050A03637075


In [24]:
import binascii
binary_string = binascii.unhexlify(hex_str)
obj = sy.deserialize(binary_string, from_bytes=True)
obj

tensor([ 6, 12, 18])